# Exploratory Data Analysis

Lyft 3D Object Detection Dataset

# Install Lyft Dataset SDK

In [ ]:
!pip install lyft_dataset_sdk

In [ ]:
# Make Paths Compatible
import os
os.system('rm -f data && ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_data data')
os.system('rm  -f images && ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_images images')
os.system('rm  -f maps && ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_maps maps')
os.system('rm  -f lidar && ln -s /kaggle/input/3d-object-detection-for-autonomous-vehicles/train_lidar lidar')

In [ ]:
# Load the SDK
%matplotlib inline
from lyft_dataset_sdk.lyftdataset import LyftDataset

level5data = LyftDataset(data_path = './', json_path = './data', verbose = True)

# SDK Functions for Data Exploration

In [ ]:
level5data.list_scenes()

In [ ]:
example_scene = level5data.scene[0]
example_scene

In [ ]:
sample_token = example_scene["first_sample_token"]
level5data.render_sample(sample_token)

In [ ]:
sample = level5data.get('sample', sample_token)
sample

In [ ]:
level5data.list_sample(sample["token"])

In [ ]:
level5data.render_pointcloud_in_image(sample_token = sample["token"],
                                      dot_size = 1,
                                      camera_channel = 'CAM_BACK')

In [ ]:
sample["data"]

In [ ]:
sensor_channel = 'LIDAR_TOP'
sample_data = level5data.get('sample_data', sample['data'][sensor_channel])
sample_data

In [ ]:
level5data.render_sample_data(sample_data['token'])

In [ ]:
sensor_channel = 'CAM_BACK'
sample_data = level5data.get('sample_data', sample['data'][sensor_channel])
sample_data

In [ ]:
level5data.render_sample_data(sample_data['token'])

In [ ]:
annotation_token = sample['anns'][4]
annotation = sample_data.get('sample_annotation', annotation_token)
annotation

In [ ]:
level5data.render_annotation(annotation_token)

In [ ]:
instance = level5data.instance[0]
instance

In [ ]:
instance_token = instance['token']
level5data.render_instance(instance_token)

In [ ]:
print("First annotated sample of this instance:")
level5data.render_annotation(instance['first_annotation_token'])

In [ ]:
print("Last annotated sample of this instance:")
level5data.render_annotation(instance["last_annotation_token"])

In [ ]:
level5data.list_categories()

In [ ]:
level5data.category[0]

In [ ]:
level5data.list_attributes()

In [ ]:
for instance in level5data.instance:
    first_token = instance['first_annotation_token']
    last_token = instance['last_annotation_token']
    nbr_samples = instance['nbr_annotations']
    current_token = first_token
    
    i = 0
    found_change = False
    while current_token != last_token:
        current_ann = level5data.get('sample_annotation', current_token)
        current_attr = level5data.get('attribute', current_ann['attribute_tokens'][0])['name']
        
        if i == 0:
            pass
        elif current_attr != last_attr:
            print(f"Changed from {last_attr} to {current_attr} at timestamp {i} out of {nbr_samples}")
            found_change = True
            
        next_token = current_ann["next"]
        current_token = next_token
        last_attr = current_attr
        i += 1

In [ ]:
level5data.sensor

In [ ]:
level5data.sample_data[10]

In [ ]:
level5data.calibrated_sensor[10]

In [ ]:
level5data.ego_pose[10]

In [ ]:
print(f"Number of logs loaded in database: {len(level5data.log)}")

In [ ]:
level5data.log[0]

In [ ]:
print(f"There are {len(level5data.map)} maps masks in the loaded dataset")

In [ ]:
level5data.map[0]

In [ ]:
sensor_channel = 'LIDAR_TOP'
sample_data = level5data.get('sample_data', sample['data'][sensor_channel])
level5data.render_sample_data(sample_data['token'])

In [ ]:
sensor_channel = 'CAM_FRONT'
sample_data = level5data.get('sample_data', sample['data'][sensor_channel])
level5data.render_sample_data(sample_data['token'])

In [ ]:
level5data.render_sample_data(sample['data']['LIDAR_TOP'], nsweeps=5)

# Videos

In [ ]:
from matplotlib import animation, rc
import matplotlib.pyplot as plt
from IPython.core.display import HTML

In [ ]:
cams = ["CAM_FRONT", "CAM_FRONT_RIGHT", "CAM_FRONT_LEFT",
        "CAM_BACK", "CAM_BACK_RIGHT", "CAM_BACK_LEFT"]

def generate_next_token(scene):
    scene = level5data.scene[scene]
    sample_token = scene['first_sample_token']
    sample_record = level5data.get('sample', sample_token)
    
    while sample_record["next"]:
        sample_token = sample_record["next"]
        sample_record = level5data.get("sample", sample_token)
        
        yield sample_token
        
def animate_images(scene, frames, pointsensor_channel = "LIDAR_TOP", interval = 1):
    generator = generate_next_token(scene)

    fig, axs = plt.subplots(
        2, len(cams), figsize=(3*len(cams), 6), 
        sharex=True, sharey=True, gridspec_kw = {'wspace': 0, 'hspace': 0.1}
    )
    
    plt.close(fig)

    def animate_fn(i):
        for _ in range(interval):
            sample_token = next(generator)
            
        for c, camera_channel in enumerate(cams):    
            sample_record = level5data.get("sample", sample_token)

            pointsensor_token = sample_record["data"][pointsensor_channel]
            camera_token = sample_record["data"][camera_channel]
            
            axs[0, c].clear()
            axs[1, c].clear()
            
            level5data.render_sample_data(camera_token, with_anns=False, ax=axs[0, c])
            level5data.render_sample_data(camera_token, with_anns=True, ax=axs[1, c])
            
            axs[0, c].set_title("")
            axs[1, c].set_title("")

    anim = animation.FuncAnimation(fig, animate_fn, frames=frames, interval=interval)
    return anim

In [ ]:
anim = animate_images(scene=3, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim = animate_images(scene=4, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
def animate_lidar(scene, frames, pointsensor_channel = "LIDAR_TOP", interval = 1):
    generator = generate_next_token(scene)
    
    fig, axs = plt.subplots(2, 1, figsize = (16, 8))
    plt.close(fig)
    
    def animate_function(i):
        for _ in range(interval):
            sample_token = next(generator)
            
        axs[0].clear()
        axs[1].clear()
        
        sample_record = level5data.get("sample", sample_token)
        pointsensor_token = sample_record["data"][pointsensor_channel]
        
        level5data.render_sample_data(pointsensor_token, with_anns=False, ax=axs[0])
        level5data.render_sample_data(pointsensor_token, with_anns=True, ax=axs[1])
        
    anim = animation.FuncAnimation(fig, animate_function, frames=frames, interval=interval)
    return anim

In [ ]:
anim = animate_lidar(scene=5, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))

In [ ]:
anim = animate_lidar(scene=10, frames=100, interval=1)
HTML(anim.to_jshtml(fps=8))